In [2]:
!pip -q install smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [23]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=022d1de90cf05aef2e379a94877cb95243c07d311c1821b70b35acb29fb75f52
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps


In [31]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = "your-hf-token"  # Kendi token'ınızı buraya yazın

In [24]:
from typing import Optional, List, Dict
from smolagents import CodeAgent, HfApiModel, tool
from datetime import datetime, timedelta
import googlemaps

In [41]:
from typing import Optional
from smolagents import CodeAgent, HfApiModel, tool

@tool
def get_istanbul_travel_duration(start_location: str, destination_location: str, time_of_day: str) -> str:
    """İki konum arası seyahat süresini ve trafik durumunu hesaplar.
    Args:
        start_location: Başlangıç konumu
        destination_location: Varış konumu
        time_of_day: Saat (HH:MM formatında)
    """
    import os
    import googlemaps
    from datetime import datetime

    gmaps = googlemaps.Client(os.getenv("your-gmaps-token"))

    try:
        hour = int(time_of_day.split(":")[0])
        traffic_multiplier = 1.0
        if 7 <= hour <= 10:
            traffic_multiplier = 1.8
        elif 16 <= hour <= 20:
            traffic_multiplier = 2.0

        directions_result = gmaps.directions(
            start_location + ", Istanbul",
            destination_location + ", Istanbul",
            mode="driving",
            departure_time=datetime.now()
        )

        if not directions_result:
            return "Rota bulunamadı"

        base_duration = directions_result[0]["legs"][0]["duration"]["value"] / 60
        traffic_duration = int(base_duration * traffic_multiplier)

        traffic_level = "Normal"
        if traffic_multiplier >= 1.8:
            traffic_level = "Yoğun"
        elif traffic_multiplier >= 1.4:
            traffic_level = "Orta"

        return f"Süre: {traffic_duration} dakika, Trafik: {traffic_level}"

    except Exception as e:
        return f"Hata: {str(e)}"

@tool
def get_attraction_info(location: str) -> str:
    """Belirli bir lokasyon hakkında bilgi verir.
    Args:
        location: Bilgi alınmak istenen turistik yerin adı
    """
    attractions = {
        "Ayasofya": {
            "visit_duration": 90,
            "opening_hours": "09:00-17:00",
            "description": "Bizans döneminden kalma tarihi müze ve cami"
        },
        "Topkapı Sarayı": {
            "visit_duration": 180,
            "opening_hours": "09:00-18:00",
            "description": "Osmanlı İmparatorluğu'nun ana sarayı"
        },
        "Kapalıçarşı": {
            "visit_duration": 120,
            "opening_hours": "08:00-19:00",
            "description": "Dünyanın en büyük kapalı çarşılarından biri"
        },
        "Sultanahmet Camii": {
            "visit_duration": 60,
            "opening_hours": "08:00-19:00",
            "description": "Mavi Cami olarak da bilinen tarihi cami"
        },
        "Galata Kulesi": {
            "visit_duration": 60,
            "opening_hours": "09:00-20:00",
            "description": "Tarihi kule ve muhteşem İstanbul manzarası"
        }
    }

    if location in attractions:
        info = attractions[location]
        return f"{location}: {info['description']}. Ziyaret süresi: {info['visit_duration']} dakika. Açık saatler: {info['opening_hours']}"
    return f"Bu lokasyon hakkında bilgi bulunamadı: {location}"

@tool
def plan_day_trip(start_time: Optional[str] = "09:00") -> str:
    """İstanbul'da bir günlük gezi planı oluşturur.
    Args:
        start_time: Gezi başlangıç saati (HH:MM formatında)
    """
    locations = ["Ayasofya", "Topkapı Sarayı", "Sultanahmet Camii", "Kapalıçarşı", "Galata Kulesi"]
    current_time = start_time or "09:00"  # Eğer None gelirse varsayılan değeri kullan
    current_location = "Sultanahmet"
    plan = "İSTANBUL GÜNLÜK GEZİ PLANI\n\n"

    for loc in locations:
        travel_info = get_istanbul_travel_duration(current_location, loc, current_time)
        location_info = get_attraction_info(loc)

        plan += f"Lokasyon: {loc}\n"
        plan += f"Varış Saati: {current_time}\n"
        plan += f"Ulaşım: {travel_info}\n"
        plan += f"Bilgi: {location_info}\n\n"

        current_location = loc

        hour, minute = map(int, current_time.split(":"))
        travel_minutes = int(travel_info.split()[1])
        new_minutes = minute + travel_minutes + 60
        new_hour = (hour + new_minutes // 60) % 24
        new_minute = new_minutes % 60
        current_time = f"{new_hour:02d}:{new_minute:02d}"

        if hour >= 19:
            break

    return plan

@tool
def enhance_location_description(location_info: str) -> str:
    """Lokasyon bilgisini daha detaylı hale getirir.
    Args:
        location_info: Temel lokasyon bilgisi
    """
    prompt = f"""
    {location_info}
    Bu lokasyon için:
    1. Önerilen aktiviteler
    2. En iyi fotoğraf çekim noktaları
    3. Dikkat edilmesi gerekenler
    4. Yeme-içme önerileri
    konularında detaylı bilgi ver.
    """
    return prompt

@tool
def plan_detailed_trip(start_time: Optional[str] = "09:00") -> str:
    """İstanbul'da detaylı bir günlük gezi planı oluşturur.
    Args:
        start_time: Gezi başlangıç saati (HH:MM formatında)
    """
    # Önce temel planı oluştur
    basic_plan = plan_day_trip(start_time)
    # Sonra her lokasyon için detaylı bilgi al
    enhanced_plan = "🗺️ DETAYLI İSTANBUL GEZİ PLANI 🗺️\n\n"

    for loc in ["Ayasofya", "Topkapı Sarayı", "Sultanahmet Camii", "Kapalıçarşı", "Galata Kulesi"]:
        basic_info = get_attraction_info(loc)
        enhanced_info = enhance_location_description(basic_info)
        enhanced_plan += f"📍 {enhanced_info}\n\n"

    return enhanced_plan



# Agent'ı oluştur
agent = CodeAgent(
    tools=[get_istanbul_travel_duration, get_attraction_info, plan_day_trip],
    model=HfApiModel(),
    additional_authorized_imports=["datetime"]
)

In [43]:
result = agent.run("İstanbul'da yarın sabah 9'dan itibaren gezebileceğim detaylı bir rota oluşturur musun? Her lokasyon için öneriler de istiyorum.")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ İstanbul'da yarın sabah 9'dan itibaren gezebileceğim detaylı bir rota oluşturur musun? Her lokasyon için        │
│ öneriler de istiyorum.                                                                                          │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  start_time = "09:00"                                                                                             
  trip_plan = plan_day_trip(start_time=start_time)                                                                 
  print(trip_plan)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Must provide API key or enterprise credentials when creating client.

[Step 0: Duration 0.04 seconds| Input tokens: 2,284 | Output tokens: 80]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Detailed itinerary with manual information and recommendations:\n\n1. 9:00 AM - Blue Mosque        
  (Sultanahmet)\n   Information: The Sultan Ahmed Mosque, also known as the Blue Mosque, is one of the most        
  famous mosques in the world. It is known for its six minarets and its vast collection of blue and white Iznik    
  tiles.\n   Recommendation: Visit the mosque in the early morning to avoid crowds and take a guided tour to       
  learn more about its history and architecture.\n\n2. 11:00 AM - Hagia Sophia\n   Information: Hagia Sophia is a  
  former Orthodox patriarchal basilica, later an Ottoman imperial mosque, and now a museum in Istanbul, Turkey.    
  It is famous for its massive dome, intricate mosaics, and historical significance.\n   Recommendation: Explore   
  the interior of Hagia Sophia and admire the stunning mosaics and architecture. Visit the museum shop to buy      
  souvenirs.\n\n3. 1:00 PM - Lunch at a local restaurant in Sultanahmet\n   Information: Sultanahmet is a          
  bustling area with many traditional Turkish restaurants.\n   Recommendation: Try a local restaurant for a        
  traditional Turkish lunch, such as kebabs, mezes, and Turkish coffee.\n\n4. 2:00 PM - Topkapi Palace\n           
  Information: Topkapi Palace is the former residence and administrative center of the Ottoman Sultans. It is now  
  a museum and a UNESCO World Heritage Site.\n   Recommendation: Visit the palace in the afternoon to see the      
  Harem, the Treasury, and the Imperial Pavilion. Take a guided tour to learn more about the Ottoman               
  Empire.\n\n5. 4:00 PM - Spice Bazaar (Egyptian Bazaar)\n   Information: The Spice Bazaar, also known as the      
  Egyptian Bazaar, is a historic covered market in Istanbul. It is famous for its colorful spices, textiles, and   
  souvenirs.\n   Recommendation: Haggling is a must in the Spice Bazaar. Try to negotiate the prices and buy some  
  souvenirs to take home.\n\n6. 6:00 PM - Dinner at a local restaurant in Beyoğlu\n   Information: Beyoğlu is a    
  vibrant neighborhood in Istanbul known for its nightlife and traditional Turkish and international cuisine.\n    
  Recommendation: Try a local restaurant in Beyoğlu for a dinner with a view. Some popular options include         
  restaurants near the Galata Tower or along the Bosphorus.\n\n7. 8:00 PM - Istiklal Avenue for shopping and       
  entertainment\n   Information: Istiklal Avenue is one of the most famous and busiest streets in Istanbul. It is  
  known for its cafes, bars, restaurants, and shops.\n   Recommendation: Walk along Istiklal Avenue and explore    
  the many cafes, bars, and shops. Enjoy the nightlife and try some local drinks.")                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Detailed itinerary with manual information and recommendations:

1. 9:00 AM - Blue Mosque (Sultanahmet)
   Information: The Sultan Ahmed Mosque, also known as the Blue Mosque, is one of the most famous mosques in the 
world. It is known for its six minarets and its vast collection of blue and white Iznik tiles.
   Recommendation: Visit the mosque in the early morning to avoid crowds and take a guided tour to learn more about
its history and architecture.

2. 11:00 AM - Hagia Sophia
   Information: Hagia Sophia is a former Orthodox patriarchal basilica, later an Ottoman imperial mosque, and now a
museum in Istanbul, Turkey. It is famous for its massive dome, intricate mosaics, and historical significance.
   Recommendation: Explore the interior of Hagia Sophia and admire the stunning mosaics and architecture. Visit the
museum shop to buy souvenirs.

3. 1:00 PM - Lunch at a local restaurant in Sultanahmet
   Information: Sultanahmet is a bustling area with many traditional Turkish restaurants.
   Recommendation: Try a local restaurant for a traditional Turkish lunch, such as kebabs, mezes, and Turkish 
coffee.

4. 2:00 PM - Topkapi Palace
   Information: Topkapi Palace is the former residence and administrative center of the Ottoman Sultans. It is now 
a museum and a UNESCO World Heritage Site.
   Recommendation: Visit the palace in the afternoon to see the Harem, the Treasury, and the Imperial Pavilion. 
Take a guided tour to learn more about the Ottoman Empire.

5. 4:00 PM - Spice Bazaar (Egyptian Bazaar)
   Information: The Spice Bazaar, also known as the Egyptian Bazaar, is a historic covered market in Istanbul. It 
is famous for its colorful spices, textiles, and souvenirs.
   Recommendation: Haggling is a must in the Spice Bazaar. Try to negotiate the prices and buy some souvenirs to 
take home.

6. 6:00 PM - Dinner at a local restaurant in Beyoğlu
   Information: Beyoğlu is a vibrant neighborhood in Istanbul known for its nightlife and traditional Turkish and 
international cuisine.
   Recommendation: Try a local restaurant in Beyoğlu for a dinner with a view. Some popular options include 
restaurants near the Galata Tower or along the Bosphorus.

7. 8:00 PM - Istiklal Avenue for shopping and entertainment
   Information: Istiklal Avenue is one of the most famous and busiest streets in Istanbul. It is known for its 
cafes, bars, restaurants, and shops.
   Recommendation: Walk along Istiklal Avenue and explore the many cafes, bars, and shops. Enjoy the nightlife and 
try some local drinks.

[Step 2: Duration 0.05 seconds| Input tokens: 7,808 | Output tokens: 1,619]

Detailed itinerary with manual information and recommendations:

1. 9:00 AM - Blue Mosque (Sultanahmet)
   Information: The Sultan Ahmed Mosque, also known as the Blue Mosque, is one of the most famous mosques in the world. It is known for its six minarets and its vast collection of blue and white Iznik tiles.
   Recommendation: Visit the mosque in the early morning to avoid crowds and take a guided tour to learn more about its history and architecture.

2. 11:00 AM - Hagia Sophia
   Information: Hagia Sophia is a former Orthodox patriarchal basilica, later an Ottoman imperial mosque, and now a museum in Istanbul, Turkey. It is famous for its massive dome, intricate mosaics, and historical significance.
   Recommendation: Explore the interior of Hagia Sophia and admire the stunning mosaics and architecture. Visit the museum shop to buy souvenirs.

3. 1:00 PM - Lunch at a local restaurant in Sultanahmet
   Information: Sultanahmet is a bustling area with many traditional Turkish restaur